In [10]:
!pip install -U langchain langchain-community langchain-core \
  langchain-text-splitters chromadb \
  sentence-transformers transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
import os
import torch

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# trandformers 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline as hf_pipeline
from langchain_community.llms import HuggingFacePipeline


# chargement des pdfs 

In [12]:
def load_pdfs(pdf_paths):
    """
    Charge plusieurs fichiers PDF et renvoie une liste de Documents.
    Chaque page du PDF devient un Document (page_content + metadata).
    """
    all_docs = []
    for path in pdf_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Fichier introuvable : {path}")
        print(f" - Chargement du PDF : {path}")
        loader = PyPDFLoader(path)
        docs = loader.load()
        print(f"   -> {len(docs)} pages chargées")
        all_docs.extend(docs)
    print(f"\nTotal : {len(all_docs)} pages sur 1 ensemble des PDFs\n")
    return all_docs

# decoupage en chunks 

In [13]:
def split_documents(docs):
    """
    Découpe les Documents (pages PDF) en chunks plus petits.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,        # taille max d'un chunk (en caractères)
        chunk_overlap=200      # recouvrement
    )
    chunks = splitter.split_documents(docs)
    print(f"{len(chunks)} chunks créés après découpage\n")
    return chunks

# embbeding +chroma

In [14]:
def build_vectorstore(chunks, persist_directory: str = "chroma_persist_db"):
    """
    Crée une base vectorielle Chroma à partir des chunks.
    Utilise des embeddings SentenceTransformers.
    """
    print("Construction de la base vectorielle (embeddings + Chroma)…")

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    print("Vectorstore prêt \n")
    return vectorstore

# prompt rag tres strict 

In [15]:
RAG_PROMPT_TEMPLATE = """

Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MEME langue que la QUESTION
   (si la question est en français alors reponse en français,
   si elle est en anglais alors reponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l’information n’est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
{context}
QUESTION_DE_L_UTILISATEUR =
{question}
R_PONSE (une seule langue, celle de la question) :
"""

def build_prompt():
    return PromptTemplate(
        input_variables=["context", "question"],
        template= RAG_PROMPT_TEMPLATE
    )

def format_docs(docs):
    """
    Concatène le contenu des Documents en une seule chaîne de contexte.
    """
    return "\n\n".join(doc.page_content for doc in docs)

# llm open source fort 

In [16]:
def build_llm(model_name: str = "Qwen/Qwen2.5-7B-Instruct"):
    """
    Construit un LLM open source fort (Qwen2.5-7B-Instruct)
    via Transformers.
    Nécessite une bonne machine GPU.
    """
    print(f"Chargement du modèle : {model_name} ...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    text_gen = hf_pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=False,
        repetition_penalty=1.05
    )

    llm = HuggingFacePipeline(pipeline=text_gen)
    print("LLM Qwen2.5-7B prêt \n")
    return llm

# chaine rag

In [17]:
def build_rag_chain(vectorstore, llm):
    """
    Construit une chaîne RAG: question -> récupérer -> formatage du contexte ->
    prompt -> LLM
    """
    retrieved = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}  # nombre de chunks récupérés
    )

    prompt = build_prompt()

    rag_chain = (
        {
            "context": retrieved | format_docs,
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
    )

    return rag_chain

def answer_question(rag_chain, question: str) -> str:
    """
    Appelle la chaîne RAG sur une question donnée.
    """
    return rag_chain.invoke(question)

# main : 

In [ ]:
# MAIN

print("=== RAG LangChain + Qwen2.5-7B-Instruct + 2 PDF ===\n")

# 1) Charger les PDF
PDF_PATHS = [
    "/kaggle/input/pdf1-chapitre1/Chapitre1.pdf" ,
    "/kaggle/input/pdf2-intronlp/IntroNLP.pdf"
]
docs = load_pdfs(PDF_PATHS)

# 2) Découper en chunks
chunks = split_documents(docs)

# 3) Construire la base vectorielle
vectorstore = build_vectorstore(chunks)

# 4) LLM
llm = build_llm()

# 5) Chaîne RAG
rag_chain = build_rag_chain(vectorstore, llm)

print("Tes deux PDF sont indexés.")

print("Tape 'exit' pour quitter.\n")

while True:
    try:
        question = input(">>> Question :  ").strip()
    except (EOFError, KeyboardInterrupt):
        print("\nAu revoir")
        break

    if question.lower() in {"exit", "quit"}:
        print("Au revoir")
        break

    if not question:
        continue

    print("\n[Assistant] Réponse en cours...\n")
    try:
        response = answer_question(rag_chain, question)
        print(response)
        print("\n" + "-" * 60 + "\n")
    except Exception as e:
        print("Erreur lors de la génération de la réponse :", e)
        print("\n" + "-" * 60 + "\n")

# Extrait pour gestion d’erreur séparé (à ajouter si besoin)

=== RAG LangChain + Qwen2.5-7B-Instruct + 2 PDF ===

 - Chargement du PDF : /kaggle/input/pdf1-chapitre1/Chapitre1.pdf
   -> 35 pages chargées
 - Chargement du PDF : /kaggle/input/pdf2-intronlp/IntroNLP.pdf
   -> 37 pages chargées

Total : 72 pages sur 1 ensemble des PDFs

76 chunks créés après découpage

Construction de la base vectorielle (embeddings + Chroma)…
Vectorstore prêt 

Chargement du modèle : Qwen/Qwen2.5-7B-Instruct ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


LLM Qwen2.5-7B prêt 

Tes deux PDF sont indexés.
Tape 'exit' pour quitter.



>>> Question :   C'est quoi NLP ?



[Assistant] Réponse en cours...



Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MEME langue que la QUESTION
   (si la question est en français alors reponse en français,
   si elle est en anglais alors reponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l’information n’est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
C’est quoi NLP
2
Le Traitement Automatique du Langage Naturel (TALN) ou Natural Language Processing (NLP) est 
une branche de l’intelligence artificielle qui vise à permettre aux machines de comprendre, interpréter, 
générer et interagir en langage humain, qu’il soit écrit ou oral.

C’est quoi NLP
2
Le Traitement Automatique du Langage Naturel (TALN) ou Natura

>>> Question :   C'est quoi LLM ?



[Assistant] Réponse en cours...



Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MEME langue que la QUESTION
   (si la question est en français alors reponse en français,
   si elle est en anglais alors reponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l’information n’est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
Chapitre 1 : 
Représentation des 
textes
01
02
03
04

Chapitre 1 : 
Représentation des 
textes
01
02
03
04

22
Phase d’apprentissage d’un modèle 
Transformer / LLM
Les LLMs commencent par être entraînés sur des corpus massifs et variés, contenant :
• des livres, 
• articles scientifiques, 
• sites web (Wikipedia, Reddit, blogs, forums),
• des données multiling